# 基础部分

## 入门

### 第一个例子——线性回归

In [1]:
import tensorflow as tf
import numpy as np

# create data
x_data = np.random.rand(100).astype(np.float32)
y_data = x_data*0.1+0.3

In [2]:
# create structure
Weights = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
biases = tf.Variable(tf.zeros([1]))
y = Weights*x_data+biases

loss = tf.reduce_mean(tf.square(y-y_data))
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

In [5]:
# run!
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for step in range(200):
    sess.run(train)
    if step % 20 == 0:
        print(step, sess.run(Weights), sess.run(biases))

0 [0.56654555] [0.04956031]
20 [0.24322964] [0.22000338]
40 [0.144382] [0.27521178]
60 [0.11375248] [0.29231897]
80 [0.10426144] [0.2976199]
100 [0.10132046] [0.2992625]
120 [0.10040917] [0.2997715]
140 [0.10012682] [0.2999292]
160 [0.1000393] [0.29997805]
180 [0.10001219] [0.29999322]


### 第二个例子——官方文档

In [1]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
#转化为浮点数
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Epoch 1/5
60000/60000 [==============================] - 13s 210us/sample - loss: 0.2901 - acc: 0.9170
Epoch 2/5
60000/60000 [==============================] - 10s 174us/sample - loss: 0.1396 - acc: 0.9583
Epoch 3/5
60000/60000 [==============================] - 10s 173us/sample - loss: 0.1075 - acc: 0.9678
Epoch 4/5
60000/60000 [==============================] - 10s 167us/sample - loss: 0.0881 - acc: 0.9728
Epoch 5/5
10000/10000 [==============================] - 1s 99us/sample - loss: 0.0740 - acc: 0.9759


[0.07401799967540428, 0.9759]

### Session的控制

In [1]:
# 矩阵相乘
import tensorflow as tf
matrix1 = tf.constant([[3, 3]])
matrix2 = tf.constant([[2], [2]])
product = tf.matmul(matrix1, matrix2)
# 等价于numpy的np.dot!

# method 1
sess = tf.Session()
result = sess.run(product)
print(result)
sess.close()

# method 2
with tf.Session() as sess:
    result2 = sess.run(product)
    print(result2)

# method 3
sess = tf.InteractiveSession()
print(product.eval())
sess.close()

[[12]]
[[12]]
[[12]]


### Variable的使用

In [1]:
import tensorflow as tf
isess = tf.InteractiveSession()
state = tf.Variable(0, name="counter")
print(state.name)

counter:0


In [17]:
one = tf.constant(1)
new_value = tf.add(state, one)
update = tf.assign(state, new_value)
# 初始化变量！重要 使用交互式环境也需要直接初始化，每次初始化会回到最原始的值
init = tf.global_variables_initializer()
isess.run(init)

In [19]:
for _ in range(5):
    update.eval()
    print(state.eval())

11
12
13
14
15
16
17
18
19
20


### Feeds 传入值

In [3]:
import tensorflow as tf
sess = tf.InteractiveSession()
# 可以给定数据类型和结构
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)

output = tf.multiply(input1, input2)
print(sess.run(output, feed_dict={input1: [7.0], input2: [2.0]}))

[14.]


## 模型建立

### 激励函数

##### 激励函数要做出选择：循环神经网络常用relu或tanh，卷积神经网络常用relu

#####  激励函数要求是可微分的，才能够参与反向传播

##### 不能任意选择，否则可能出现梯度消失或梯度爆炸

### 添加层

In [6]:
import tensorflow as tf
import numpy as np


def add_layer(inputs, in_size, out_size, activation_function=None):
    Weights = tf.Variable(tf.random_normal([in_size, out_size]))
    biases = tf.Variable(tf.zeros([1, out_size]+0.1))
    Wx_plus_b = tf.matmul(inputs, Weights)+biases
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    return outputs

In [7]:
x_data = np.linspace(-1.0,1.0, 300)[:, np.newaxis]
noise = np.random.normal(0, 0.05, x_data.shape)
y_data = np.square(x_data)-0.5+noise

xs=tf.placeholder([None,1])
ys=tf.placeholder([None,1])

TypeError: Expected DataType for argument 'dtype' not [None, 1].

In [11]:
l1 = add_layer(x_data, 1, 10, activation_function=tf.nn.relu)
prediction = add_layer(l1, 10, 1, activation_function=None)

loss = tf.reduce_mean(tf.reduce_sum(
    tf.square(y_data-prediction), reduction_indices=[1]))

train_step = tf.train.GradientDescentOptimazer(0.1).minimize(loss)

init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init)

for i in range(1000):
    # 
    sess.run(train_step, feed_dict: {xs: x_data, ys: y_data})

SyntaxError: invalid syntax (<ipython-input-11-2111cf8a8aeb>, line 13)